In [ ]:
##EXPRESSION MATRIX

import tqdm as tqdm
import json

datadictionary = {}
genes = []
cell_ids = []

with open('/Users/julianzulueta/Desktop/CellGeneand Expression.txt','r') as f:
    for line in f:
        for word in line.split():
           genes.append(word)
           break         
del genes[0]


with open('/Users/julianzulueta/Desktop/CellGeneand Expression.txt','r') as f:
    for line in f:
        for word in line.split():
           try:
                int(word)
                break
           except:
                cell_ids.append(word)
        break
del cell_ids[0]


from tqdm import tqdm
Names = []
for line in open('/Users/julianzulueta/Desktop/CellGeneand Expression.txt','r').readlines():
    Names.append(line.strip())

ranges = (len(Names))


listbb = [10, 100, 1000, 2000, 5532, 7363, 4430, 10458, 1825, 3317, 1912, 1733, 6196, 1928, 8448, 10825]

giantmatrix = []
for i in tqdm(listbb): #20 genes
    words_list = []
    for num_str in Names[i].split():
        try:

            float(num_str)
            words_list.append(round(float(num_str), 1))
        except:
            pass
    giantmatrix.append(words_list)
    
#del giantmatrix[0]


##indexes of interest, paper mentions that G6 and G11 were both enriched for genes linked to T cell exhaustion
print(genes.index('LAG3'), 'LAG3')
print(genes.index('PDCD1'), 'PDCD1')
print(genes.index('HAVCR2'), 'HAVCR2')
print(genes.index('TIGIT'), 'TIGIT')
print(genes.index('CD38'), 'CD38')
print(genes.index('ENTPD1'), 'ENTPD1')


##indexes of interest, paper mentions that G11 were also enriched for cell cycle genes
print(genes.index('CDK1'), 'CDK1')
print(genes.index('CCNB1'), 'CCNB1')
print(genes.index('MKI67'), 'MKI67')
print(genes.index('CDK4'), 'CDK4')
print(genes.index('RB1'), 'RB1')
print(genes.index('TP53'), 'TP53')


datadictionary["rows"] = genes[10], genes[100], genes[1000], genes[2000], genes[5532], genes[7363], genes[4430], genes[10458], genes[1825], genes[3317], genes[1912], genes[1733], genes[6196], genes[1928], genes[8448], genes[10825]
datadictionary["cols"] = cell_ids
datadictionary["matrix"] = giantmatrix

           
with open("expressionmatrix13.json", "w") as outfile:
    json.dump(datadictionary, outfile)
    


In [ ]:
### t-SNE
from tqdm import tqdm
import json

cell_dict = {}

#Automatically generate 11 empty lists for the different cell types. (Example, set_1 = [], set_2 = []....)
for i in range(1,12):       # for global vars
    globals()["list_G" + str(i)] = []

with open('/Users/julianzulueta/Desktop/t-SNE for melanoma') as topo_file:
    for line in tqdm(topo_file):
        
        cell_dict[f"{line.split()[0]}"] = {}
        try:
            cell_dict[f"{line.split()[0]}"]['mappings']['t-SNE'] = ([float(line.split()[1]), float(line.split()[2])])
        
        except:
             pass
                         
        words = 'a'       
        cell_category_replacement = {'G01': 'B cells', 
                                     'G02': 'Plasma cells',
                                     'G03': 'Monocytes/Macrophages',
                                    'G04': 'Dendritic cells',
                                    'G05': 'Lymphocytes',
                                    'G06': 'Exhausted CD8 T cells',
                                    'G07': 'Regulatory T cells',
                                    'G08': 'Cytotoxicity (Lymphocytes)',
                                    'G09': 'Exhausted/HS CD8 T cells',
                                    'G10': 'Memory T cells',
                                    'G11': 'Lymphocytes exhausted/cell-cycle'}
        
        try:
            globals()[f"list_{line.split()[3]}"].append(line.split()[0])
            cell_dict[f"{line.split()[0]}"]['factors']['cluster'] = cell_category_replacement[line.split()[3]]
        except:
            pass

(cell_dict.pop('NAME', None))
(cell_dict.pop('TYPE', None))

with open("tsne_data.json", "w") as outfile:
    json.dump(cell_dict, outfile)

In [ ]:
##Cell Sets Tree
import pandas as pd

cellsets_dict = {}

df = pd.read_excel('/Users/julianzulueta/Desktop/NIHMS1510803-supplement-10.xlsx', sheet_name='Gene marker-Fig1B-C')
cell_types = [] #Get the cell category groups (i.e. B cells, Plasma cells, Dendritic cells, etc.)
for cell in df:
    if "Unnamed" in cell: #do not include empty boxes
        pass
    else:
        cell_types.append(cell[4:]) #removes the G1- , G2- , ... before each category name

dic_set_array = []
#Automatically establish dictionaries
for i in range(len(cell_types)):     
    
    #makes a dictionary (example being dic_set_1 = {})
    globals()["dic_set_" + str(i+1)] = {} 
    
    #establishes the cell type (example is dic_set_1['names'] = 'B cells')
    globals()["dic_set_" + str(i+1)]['names'] = cell_types[i] 
    
    #adds the corresponding cells that are part of that. Example set_1 = [C0001, C0002] is all the cells that are B cells.
    #This adds dic_set_1['set'] = ["C00001", "C00002"]
    globals()["dic_set_" + str(i+1)]["set"] = (globals()["list_G" + str(i+1)]) 
    
    #append all the dictionaries together. (Example dic_set_array = dic_set_1, dic_set_2, dic_set_3...)
    dic_set_array.append((globals()["dic_set_" + str(i+1)])) 

cellsets_dict['version'] = '0.1.2'
cellsets_dict['datatype'] = 'cell'
cellsets_dict['tree'] = [{
   "name": "Cell Type Annotations",
   "children": dic_set_array}]

with open("cellsets_dataq.json", "w") as outfile:
    json.dump(cellsets_dict, outfile)